In [ ]:
!pip install transformers==4.5.1

In [2]:
import os
import transformers
import torch
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import time 
import os

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [11]:
sentencePath='/content/drive/My Drive/MTP/Datasets/english-sentences-1000.txt'
parent_dir="/content/drive/My Drive/HeatMapsReport/"

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
#This cell creates 144 folders in the specified path, 1 corresponding to each head.

for i in range(1,145):
  path=os.path.join(parent_dir,"head_"+str(i))
    
  os.mkdir(path)

In [10]:
MAX_LEN=64
BATCH_SIZE=8

tokenizer=transformers.BertTokenizer.from_pretrained('bert-base-multilingual-cased')

class SentenceDataset(Dataset):
  
  def __init__(self, sentences,tokenizer,max_len):
    self.sentences=sentences
    self.tokenizer=tokenizer
    self.max_len=max_len

  def __len__(self):
    return len(self.sentences)

  def __getitem__(self,item):
    sentence=str(self.sentences[item])

    encoding=tokenizer.encode_plus(
        sentence,
        max_length=self.max_len,
        add_special_tokens=True,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_token_type_ids=False,
        truncation=True,
        return_tensors='pt'
    )

    return{
        'sentence':sentence,
        'input_ids':encoding['input_ids'],
        'attention_mask':encoding['attention_mask']
    }

def create_data_loader(sentences,tokenizer,max_len,batch_size):
    ds= SentenceDataset(
      sentences=sentences,
      tokenizer=tokenizer,
      max_len=max_len
    )

    return DataLoader(
      ds,
      batch_size=batch_size
    )

In [ ]:
#Code to generate the heatmaps for the first 50 sentences for each functional role and language.
in_file=open(sentencePath,'r',encoding='utf-8')
lines=in_file.readlines()[:50]
data_loader=create_data_loader(lines,tokenizer,MAX_LEN,BATCH_SIZE)

bert_model = transformers.BertModel.from_pretrained('bert-base-multilingual-cased', output_attentions=True)

counti=0
plt.figure(figsize=(12, 12))
print('ENTER')
for count, data in enumerate(iter(data_loader)):
    print('HERE')
    start=time.time()
    last_hidden_state,pooler_output,attentions = bert_model(input_ids=torch.squeeze(data['input_ids']),
                                                      attention_mask=torch.squeeze(data['attention_mask']),return_dict=False)
    end=time.time()
    print('BERT TIME---',end-start)
    for attention_id, attention in enumerate(attentions):
        for element in range(attention.shape[0]):
            for head in range(attention.shape[1]):
                start=time.time()
                counti+=1
                number = attention_id * 12 + (head + 1)
                currentPath = os.path.join(parent_dir, "head_" + str(number))
                sentenceWeights = attention[element][head]
                tokens = tokenizer.tokenize(lines[count * 8 + element])
                final_tokens = ['CLS'] + tokens[:62] + ['SEP']
                sent_len = len(final_tokens)
                array = sentenceWeights[:sent_len, :sent_len].detach().numpy()
        
                #sns.set(font='Noto Sans CJK JP')

                sns.set(font_scale=2.5)
                graph = sns.heatmap(array, cmap='Blues', linewidth=1)

                graph.set_xticks(np.arange(len(final_tokens)))
                graph.set_yticks(np.arange(len(final_tokens)))

                axesLabels=[str(i+1) + ' ' + x for i,x in enumerate(final_tokens)]

                graph.set_xticklabels(axesLabels)
                graph.set_yticklabels(axesLabels)

                for item in graph.get_yticklabels():
                    item.set_rotation(0)

                for item in graph.get_xticklabels():
                    item.set_rotation(90)

                plt.savefig(currentPath + '/sentence_' + str(count * 8 + element + 1) + '.png',bbox_inches = "tight")
                plt.clf()
                end=time.time()
                print('TOTAL TIME----',start-end)

                print(counti)